### REDDIT API Connexion

In [1]:
import requests
import pandas as pd
import csv
import time
from datetime import datetime
import os
from secret import CLIENT_ID, SECRET, USER, PASSWORD

In [2]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET)

In [3]:
data = {
    'grant_type' : 'password', 
    'username' : USER, # a modifier
    'password' : PASSWORD # a modifier
}

In [5]:
headers = {'User-Agent': 'MyAPI/0.0.1'}
res  = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers = headers)
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

## Récupération des comptes d'un SubReddit

In [11]:
params = {
    'limit': 15000,  # Limite le nombre de résultats à 100 par page
    'after': None  # Permet de récupérer les résultats suivants
}

In [4]:

def recupAccounts(subName: str) -> pd.DataFrame:
    """
    Récupère les auteurs des posts d'un subreddit
    
    :param subName: nom du subreddit
    :return: DataFrame contenant les auteurs des posts
    """

    i = 0
    df = pd.DataFrame()
    url = f'https://oauth.reddit.com/r/{subName}/hot'
    while True:
        response = requests.get(url, headers=headers, params=params)
        i +=1
        print(i)

        if response.status_code == 200:
            data = response.json()
            for post in data['data']['children']:
                new_row = pd.DataFrame({'author': [post['data']['author']]})
                df = pd.concat([df, new_row], ignore_index=True)

            
            params['after'] = data['data']['after']

            # Arrêtez la boucle si 'after' est None, ce qui signifie qu'il n'y a plus de résultats à récupérer
            if params['after'] is None:
                break
            
    return df

In [13]:
TopSubReddit = pd.read_csv('C:/Users/samma/Desktop/ultimatelist.csv') # CSV des SubReddit les plus populaires 

### Récupération des SubReddit d'un compte

In [5]:
def recupSubReddit(username: str) -> pd.DataFrame:
    """
    Récupère les subreddits où un utilisateur a posté.
    Cette fonction prend en entrée le nom d'utilisateur d'un utilisateur Reddit et renvoie un DataFrame contenant les subreddits où cet utilisateur a posté.
    
    :param username: Nom de l'utilisateur Reddit.
    :type username: str
    :return: DataFrame contenant les subreddits où l'utilisateur a posté.
    :rtype: pd.DataFrame
    """

    dfSub = pd.DataFrame()
    after = None
    headers = {'User-Agent': 'MyAPI/0.0.1'}

    # Récupère les subreddits où l'utilisateur a posté
    while True:
        url = f'https://www.reddit.com/user/{username}/submitted.json'
        params = {'after': after} if after else {}
        response = requests.get(url, headers=headers, params=params)

        # Si la requête est réussie, ajoutez les subreddits à dfSub
        if response.status_code == 200:
            submissions = response.json()['data']['children']
            for submission in submissions:
                new_row = pd.DataFrame({username: [submission['data']['subreddit']]})
                dfSub = pd.concat([dfSub, new_row], ignore_index=True)

            after = response.json()['data']['after']
            if not after:
                break
        
        # Si l'utilisateur n'existe pas, imprimez un message et arrêtez la boucle
        elif response.status_code == 404:
            print(f"Utilisateur {username} non trouvé.")
            break

        # Si le code d'état est 429, attendez 20 secondes avant de réessayer
        elif response.status_code == 429:
            wait_time = 20  # Ajoute un délai d'attente de 20 sec pour la réinitialisation
            print(f"Erreur 429 - Trop de requêtes. Attendez {wait_time} secondes avant de réessayer.")
            time.sleep(wait_time)

        else:
            print(f"La requête a échoué avec le code d'état {response.status_code}")
            break

    return dfSub


In [6]:
def recupAutoSub(accounts: pd.DataFrame) -> pd.DataFrame:
    """
    Récupère les subreddits où une liste d'utilisateurs a posté.
    
    :param accounts: DataFrame contenant les auteurs des posts
    :return: DataFrame contenant les subreddits où les utilisateurs ont posté
    """
    dfs = []  # Liste pour stocker les DataFrames des subreddits de chaque utilisateur
    i = 0  # Compteur pour suivre le nombre d'utilisateurs traités

    for compte in accounts.itertuples(index=False):
        print(i)
        i += 1
        username = compte.author

        # Vérifie que le nom d'utilisateur n'est pas '[deleted]' et n'est pas vide
        if username != '[deleted]' and username:
            # Récupère les subreddits de l'utilisateur et supprime les doublons
            df_sub = recupSubReddit(username).drop_duplicates()
            dfs.append(df_sub)

    # Concatène tous les DataFrames des subreddits en un seul DataFrame
    df_result = pd.concat(dfs, axis=1)
    return df_result



In [8]:
# fonction prinipale 

def mainFonct(nbSub: int):
    """
    Fonction principale pour récupérer les subreddits où les utilisateurs ont posté.
    Args:
        nbSub (int): The number of subreddits to process.
    """
    for i in range(0, nbSub):
        print(i)

        # Remplir le nom du SubReddit de départ
        sub_name = 'france'
        print(sub_name)
        
        # Appel de la fonction recupAccounts
        df_accounts = recupAccounts(sub_name)
        
        # Affichage du DataFrame
        print(df_accounts)
        
        # Affichage de la forme du DataFrame
        print(df_accounts.shape)

        maintenant = datetime.now()
        date_heure_lisible = maintenant.strftime("%d-%m-%Y %H:%M:%S")
        print(f"\n\n\nDébut de {sub_name}\n {date_heure_lisible}\n\n\n")
        
        df_result = recupAutoSub(df_accounts)
        df_result.to_csv(f'CSV_SubReddit/{TopSubReddit["announcements"][i]}', index=False)

        maintenant = datetime.now()
        date_heure_lisible = maintenant.strftime("%d-%m-%Y %H:%M:%S")
        print(f"\n\n\Fin de {sub_name}\n {date_heure_lisible}\n\n\n")

### Création des liens et des noeuds pour GEPHI

In [12]:
def liensCsv(fichier: str, tabUsers: set, unique_links: set) -> pd.DataFrame:
    """
    Récupère les liens entre utilisateurs et subreddits à partir d'un fichier CSV.
    
    :param fichier: Chemin du fichier CSV.
    :param tabUsers: Ensemble des utilisateurs déjà traités.
    :param unique_links: Ensemble des liens uniques déjà traités.
    :return: DataFrame contenant les liens entre utilisateurs et subreddits.
    """
    df = pd.read_csv(fichier)
    nouveau_df = pd.DataFrame(columns=['Utilisateur', 'Subreddit'])

    for colonne in df.columns:
        if len(df[colonne]) > 1 and ('.' not in colonne) and (colonne != "AutoModerator"):
            subreddits = [subreddit for subreddit in df[colonne][1:] if pd.notna(subreddit)]
            if subreddits:
                paires = pd.DataFrame({'Utilisateur': colonne, 'Subreddit': subreddits})
                # Ensure only unique pairs are added
                unique_pairs = list(zip(paires['Utilisateur'], paires['Subreddit']))
                unique_pairs = [pair for pair in unique_pairs if pair not in unique_links]
                unique_links.update(unique_pairs)
                paires = pd.DataFrame(unique_pairs, columns=['Utilisateur', 'Subreddit'])
                nouveau_df = pd.concat([nouveau_df, paires], ignore_index=True)

        if colonne not in tabUsers:
            tabUsers.add(colonne)

    return (nouveau_df, tabUsers, unique_links)

In [13]:
def noeudsCsv(fichier: str, subList: set) -> pd.DataFrame:
    """
    Récupère les noeuds à partir d'un fichier CSV.
    
    :param fichier: Chemin du fichier CSV.
    :param subList: Ensemble des subreddits déjà traités.
    :return: DataFrame contenant les noeuds.
    """

    df = pd.read_csv(fichier)
    nouveau_df = pd.DataFrame(columns=['ID', 'Label', 'Type'])

    for colonne in df.columns:
        if len(df[colonne]) > 1 and ('.' not in colonne) and (colonne != "AutoModerator"):
            triplet = pd.DataFrame({'ID': [colonne], 'Label': [None], 'Type': ['Utilisateur']})
            nouveau_df = pd.concat([nouveau_df, triplet], ignore_index=True)

            for ligne in df[colonne]:
                if ligne not in subList and pd.notna(ligne):
                    triplet = pd.DataFrame({'ID': [ligne], 'Label': [ligne], 'Type': ['SubReddit']})
                    nouveau_df = pd.concat([nouveau_df, triplet], ignore_index=True)
                    subList.add(ligne)

    return (nouveau_df, subList)

In [14]:
def mainCSVFinalOpti():
    """
    Fonction principale pour créer les liens et les noeuds à partir des ficchiers CSV.
    """

    # Initialisation des listes pour stocker les noeuds et les liens
    noeuds_list = []  
    liens_list = []   
    unique_links = set()  # Ensemble pour stocker les liens uniques

    # Répertoire contenant les fichiers CSV
    repertoire_csv = "CSV_SubReddit/"
    fichiers_csv = [f"CSV_SubReddit/{f}" for f in os.listdir(repertoire_csv)]

    subList = set()  # Ensemble pour stocker les subreddits uniques
    tabUsers = set()  # Ensemble pour stocker les utilisateurs uniques

    i = 1
    # Parcours de chaque fichier CSV dans le répertoire
    for sub in fichiers_csv:
        print(i)
        print(sub)
        i += 1

        # Récupération des liens à partir du fichier CSV
        resLiens = liensCsv(sub, tabUsers, unique_links)
        liens_list.append(resLiens[0])  
        tabUsers.update(resLiens[1])  
        unique_links = resLiens[2]  

        # Récupération des noeuds à partir du fichier CSV
        resNoeuds = noeudsCsv(sub, subList)
        noeuds_list.append(resNoeuds[0])  
        subList.update(resNoeuds[1])  

    # Concaténation de tous les DataFrames de liens en un seul DataFrame
    liens = pd.concat(liens_list, ignore_index=True) 

    liens = liens.drop_duplicates()
    liens.to_csv('NSFWLiens.csv', index=False)  # Sauvegarde des liens dans un fichier CSV

    # Concaténation de tous les DataFrames de noeuds en un seul DataFrame
    noeuds = pd.concat(noeuds_list, ignore_index=True)  
    noeuds = noeuds.drop_duplicates() 

    noeuds.to_csv('NSFWNoeuds.csv', index=False)  # Sauvegarde des noeuds dans un fichier CSV

In [ ]:
mainCSVFinalOpti()

In [4]:
def nettoyage(nbmax: int):
    """
    Fonction pour nettoyer les liens et les noeuds en fonction des 3000 premiers Subreddits.

    : param nbmax: Nombre de Subreddits à traiter.
    """
    
    # Charger le DataFrame initial
    df = pd.read_csv("OptiliensTest2.csv")
    dfNoeuds = pd.read_csv("OptinoeudsTest2.csv")

    # Charger la liste des 3000 premiers Subreddits
    TopSubReddit = pd.read_csv("ultimatelist.csv", header=None, names=["id", "Subreddit", "nsfw"])

    # Filtrer les lignes en fonction du Top 3000
    df_nsfw_true = TopSubReddit[TopSubReddit['nsfw'] == 'nsfw=true']
    df_nettoyerliens = df[df['Subreddit'].isin(df_nsfw_true['Subreddit'])]

    # Récupérer les utilisateurs présents dans les liens avec "id" comme nom d'utilisateur
    utilisateurs_liens = pd.DataFrame({'ID': df_nettoyerliens['Utilisateur'], 'Label': '', 'Type': 'Utilisateur'})

    noeuds_subnbmax = pd.DataFrame({'ID': df_nettoyerliens['Subreddit'], 'Label': df_nettoyerliens['Subreddit'], 'Type': 'Subreddit'})

    # Concaténer les nœuds initiaux avec les nouveaux utilisateurs
    df_nettoyerNoeuds = pd.DataFrame()
    df_nettoyerNoeuds = pd.concat([noeuds_subnbmax, utilisateurs_liens], ignore_index=True)


    # Supprimer les doublons pour s'assurer que les utilisateurs sont uniques dans les nœuds
    df_nettoyerNoeuds = df_nettoyerNoeuds.drop_duplicates(subset='ID')

    # Sauvegarder les nouveaux nœuds
    df_nettoyerNoeuds.to_csv('nsfwNoeuds.csv', index=False)

    # Sauvegarder le nouveau DataFrame de liens
    df_nettoyerliens.to_csv('nsfwliens.csv', index=False)


In [5]:
nettoyage(1000)

In [7]:
nettoyage(500)